In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import euclidean

In [2]:
df = pd.read_csv("parkinson_5%.csv")

In [3]:
df

,Jitter (local),"Jitter (local, absolute)",Jitter (rap),Jitter (ppq5),Jitter (ddp),Shimmer (local),"Shimmer (local, dB)",Shimmer (apq3),Shimmer (apq5),Shimmer (apq11),...,Maximum pitch,Number of pulses,Number of periods,Mean period,Standard deviation of period,Fraction of locally unvoiced frames,Number of voice breaks,Degree of voice breaks,features 27,class
0,1.49,0.0,0.90,NaN,2.70,8.33,0.78,4.52,4.61,6.80,...,187.58,160.0,159.0,0.01,0.0,0.00,0.0,0.00,23.0,1.0
1,0.73,0.0,0.35,0.38,1.06,5.86,0.64,NaN,3.18,7.19,...,234.51,170.0,169.0,0.01,NaN,2.25,0.0,0.00,23.0,1.0
2,1.22,0.0,0.73,0.67,2.20,8.72,0.88,4.35,5.17,7.55,...,211.44,1431.0,1427.0,0.01,0.0,10.66,1.0,0.18,23.0,1.0
3,2.50,0.0,1.16,1.63,3.47,13.51,NaN,5.26,8.77,16.78,...,220.23,94.0,92.0,0.00,0.0,NaN,0.0,0.00,23.0,1.0
4,3.51,0.0,1.72,1.54,5.15,9.11,1.04,3.10,4.93,12.82,...,225.16,117.0,114.0,0.00,0.0,18.18,1.0,13.32,23.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1035,5.13,0.0,2.72,2.51,8.16,9.85,0.97,4.59,6.81,8.30,...,158.59,63.0,60.0,0.01,0.0,24.19,2.0,15.11,1.0,0.0
1036,1.76,NaN,NaN,0.69,2.43,7.57,0.74,2.72,NaN,11.93,...,143.80,47.0,44.0,0.01,0.0,46.05,1.0,25.79,1.0,0.0
1037,2.91,0.0,1.14,1.27,3.42,15.49,1.46,6.43,NaN,14.24,...,149.64,79.0,77.0,0.01,NaN,20.00,1.0,4.57,1.0,0.0
1038,3.79,0.0,1.79,1.48,5.36,15.70,1.37,6.37,11.09,18.82,...,451.37,75.0,70.0,0.01,0.0,NaN,3.0,19.58,1.0,0.0


In [4]:
df.dtypes

Jitter (local)                         float64
Jitter (local, absolute)               float64
Jitter (rap)                           float64
Jitter (ppq5)                          float64
Jitter (ddp)                           float64
Shimmer (local)                        float64
Shimmer (local, dB)                    float64
Shimmer (apq3)                         float64
Shimmer (apq5)                         float64
Shimmer (apq11)                        float64
Shimmer (dda)                          float64
AC                                     float64
NTH                                    float64
HTN                                    float64
Median pitch                           float64
Mean pitch                             float64
Standard deviation                     float64
Minimum pitch                          float64
Maximum pitch                          float64
Number of pulses                       float64
Number of periods                      float64
Mean period  

In [5]:
df.isnull().sum()

Jitter (local)                         56
Jitter (local, absolute)               55
Jitter (rap)                           60
Jitter (ppq5)                          52
Jitter (ddp)                           59
Shimmer (local)                        37
Shimmer (local, dB)                    54
Shimmer (apq3)                         44
Shimmer (apq5)                         71
Shimmer (apq11)                        60
Shimmer (dda)                          51
AC                                     50
NTH                                    51
HTN                                    59
Median pitch                           54
Mean pitch                             69
Standard deviation                     52
Minimum pitch                          53
Maximum pitch                          52
Number of pulses                       57
Number of periods                      46
Mean period                            56
Standard deviation of period           53
Fraction of locally unvoiced frame

In [6]:
null_counts = df.isnull().sum().sum()
total = df.size
null_percentages = 100 * null_counts / total
print("total nilai :",total)
print("nilai hilang :",null_counts)
print("persentase :",null_percentages)

total nilai : 29120
nilai hilang : 1456
persentase : 5.0


In [7]:
null_counts = df.isnull().sum()
null_percentages = 100 * null_counts / len(df)
null_percentages

Jitter (local)                         5.384615
Jitter (local, absolute)               5.288462
Jitter (rap)                           5.769231
Jitter (ppq5)                          5.000000
Jitter (ddp)                           5.673077
Shimmer (local)                        3.557692
Shimmer (local, dB)                    5.192308
Shimmer (apq3)                         4.230769
Shimmer (apq5)                         6.826923
Shimmer (apq11)                        5.769231
Shimmer (dda)                          4.903846
AC                                     4.807692
NTH                                    4.903846
HTN                                    5.673077
Median pitch                           5.192308
Mean pitch                             6.634615
Standard deviation                     5.000000
Minimum pitch                          5.096154
Maximum pitch                          5.000000
Number of pulses                       5.480769
Number of periods                      4

# Imputasi

In [8]:
def jarak(df2,col):
    euc_dist[col] = []
    euc_dist_append =euc_dist[col].append
    for row in range(df2.shape[0]):
        current = df2.index[df2.iloc[:,col] == 0].tolist()
        current = current[0]
        a = df2.iloc[current,col+1:].tolist()
        b = df2.iloc[current,:col].tolist()
        x1 = b+a
        if(row is not current):
            a=df2.iloc[row,col+1:].tolist()
            b=df2.iloc[row,:col].tolist()
            x2=b+a
        else:
            continue
        x1 = np.nan_to_num(x1)  # Mengganti nilai NaN dengan 0
        x2 = np.nan_to_num(x2)  # Mengganti nilai NaN dengan 0
        dist = euclidean(x1,x2)
        euc_dist_append([dist,df2.iloc[row,col]])
#         print(" x1 = ",x1,"\n x2 = ",x2,"\n distance = ",dist)

    lst = list(euc_dist[col])
    lst.sort()
    euc_dist[col] = lst
#     print(euc_dist[col])

In [9]:
from collections import Counter

def hitung(val, k, temp=None, i=0, counter=0):
    counts = Counter(val)
    total_count = len(counts)
#     print("jumlah val:", total_count)
    if counter >= int((k/2)+1) and total_count <= 2 :
        print("median : ",int((k/2)+1))
        return temp
    elif i is not k:
        if temp is None:
            temp = val[i]
        elif temp != val[i]:
            temp = val[i]
            counter = 0
        # print(f"temp = {temp} , counter = {counter} , i = {i} , k = {k} , median = {int((k/2)+1)}, val[i] = {val[i]}")
        return hitung(val,k,temp,i+1,counter+1)
    else:
        return None    

def Final(df,col,k):
    terdekat = []
    mean = 0
    minimum = None
    maximum = None
    fuzzy_val = []
    euc_val = []
    terdekat_append = terdekat.append
    fuzzy_append = fuzzy_val.append
    
    for val in range(k):
        terdekat_append(euc_dist[col][val])
    
    for x,y in terdekat:
        if (minimum is None or maximum is None):
            minimum = y
            maximum = y
        elif(y<minimum):
            minimum = y
        elif(y>maximum):
            maximum = y
        mean += y
        euc_val.append(y)
    mean /= len(terdekat)
    euc_val.sort()
    print(euc_val)
    
    fuzzy = hitung(euc_val,3)
    print("min : ",minimum)
    print("max : ",maximum)
    print("mean : ",mean)
    for x,y in terdekat:
        if fuzzy is None:
            fuzzy_append((y,fuzzy_triangular(y,minimum,mean,maximum)))
        else:
            fuzzy_val.append((y,fuzzy))
        
    print("fuzzy : ",fuzzy_val)
    print("fuzzy : ",fuzzy)
    
    pembilang = 0
    penyebut = 0 
    
    for nilai,bobot in fuzzy_val:
        pembilang += nilai*bobot
        penyebut += bobot
            
    print("pembilang : ", pembilang)
    print("penyebut : ",penyebut)
    
    if fuzzy is not None:
        tertimbang = fuzzy
    else:
        tertimbang = pembilang / penyebut
    
    print("rata-rata tertimbang : ",tertimbang)
    index = dummy.index[dummy.iloc[:,col] == 0].tolist()
    index = index[0]
    
    if tertimbang == 0:
        tertimbang = dummy.iloc[index-1, col]
        
    dummy.iloc[index,col] = tertimbang
    print(" ")

In [10]:
def fuzzy_triangular(val, minimum, mean, maximum):
   if val <= minimum or val >= maximum: 
        return 0.0
   elif minimum < val <= mean:
        return float((val - minimum)/(mean - minimum)) 
   elif mean < val < maximum:
        return float((maximum - val)/(maximum - mean))

In [11]:
data = df.iloc[:, :-1]
df2 = df.copy()
dummy = df2.fillna(0)
dummy.fillna(0, inplace=True)
dummy

,Jitter (local),"Jitter (local, absolute)",Jitter (rap),Jitter (ppq5),Jitter (ddp),Shimmer (local),"Shimmer (local, dB)",Shimmer (apq3),Shimmer (apq5),Shimmer (apq11),...,Maximum pitch,Number of pulses,Number of periods,Mean period,Standard deviation of period,Fraction of locally unvoiced frames,Number of voice breaks,Degree of voice breaks,features 27,class
0,1.49,0.0,0.90,0.00,2.70,8.33,0.78,4.52,4.61,6.80,...,187.58,160.0,159.0,0.01,0.0,0.00,0.0,0.00,23.0,1.0
1,0.73,0.0,0.35,0.38,1.06,5.86,0.64,0.00,3.18,7.19,...,234.51,170.0,169.0,0.01,0.0,2.25,0.0,0.00,23.0,1.0
2,1.22,0.0,0.73,0.67,2.20,8.72,0.88,4.35,5.17,7.55,...,211.44,1431.0,1427.0,0.01,0.0,10.66,1.0,0.18,23.0,1.0
3,2.50,0.0,1.16,1.63,3.47,13.51,0.00,5.26,8.77,16.78,...,220.23,94.0,92.0,0.00,0.0,0.00,0.0,0.00,23.0,1.0
4,3.51,0.0,1.72,1.54,5.15,9.11,1.04,3.10,4.93,12.82,...,225.16,117.0,114.0,0.00,0.0,18.18,1.0,13.32,23.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1035,5.13,0.0,2.72,2.51,8.16,9.85,0.97,4.59,6.81,8.30,...,158.59,63.0,60.0,0.01,0.0,24.19,2.0,15.11,1.0,0.0
1036,1.76,0.0,0.00,0.69,2.43,7.57,0.74,2.72,0.00,11.93,...,143.80,47.0,44.0,0.01,0.0,46.05,1.0,25.79,1.0,0.0
1037,2.91,0.0,1.14,1.27,3.42,15.49,1.46,6.43,0.00,14.24,...,149.64,79.0,77.0,0.01,0.0,20.00,1.0,4.57,1.0,0.0
1038,3.79,0.0,1.79,1.48,5.36,15.70,1.37,6.37,11.09,18.82,...,451.37,75.0,70.0,0.01,0.0,0.00,3.0,19.58,1.0,0.0


In [12]:
euc_dist = {}

In [13]:
for i in range(df2.shape[1]):
    print("Kolom :",i+1)
    current = dummy.index[dummy.iloc[:,i] == 0].tolist()
    for x in current:
        jarak(dummy,i)
        Final(dummy,i,3)

Kolom : 1
[1.21, 1.42, 3.09]
min :  1.21
max :  3.09
mean :  1.9066666666666665
fuzzy :  [(1.21, 0.0), (1.42, 0.3014354066985646), (3.09, 0.0)]
fuzzy :  None
pembilang :  0.4280382775119617
penyebut :  0.3014354066985646
rata-rata tertimbang :  1.42
 
[0.0, 2.08, 2.75]
min :  0.0
max :  2.75
mean :  1.61
fuzzy :  [(2.08, 0.587719298245614), (0.0, 0.0), (2.75, 0.0)]
fuzzy :  None
pembilang :  1.2224561403508771
penyebut :  0.587719298245614
rata-rata tertimbang :  2.08
 
[0.46, 0.56, 0.61]
min :  0.46
max :  0.61
mean :  0.5433333333333333
fuzzy :  [(0.56, 0.7499999999999991), (0.61, 0.0), (0.46, 0.0)]
fuzzy :  None
pembilang :  0.41999999999999954
penyebut :  0.7499999999999991
rata-rata tertimbang :  0.56
 
[1.66, 2.08, 2.83]
min :  1.66
max :  2.83
mean :  2.19
fuzzy :  [(2.83, 0.0), (1.66, 0.0), (2.08, 0.7924528301886795)]
fuzzy :  None
pembilang :  1.6483018867924535
penyebut :  0.7924528301886795
rata-rata tertimbang :  2.08
 
[0.89, 1.04, 3.6]
min :  0.89
max :  3.6
mean :  1.843

[0.0, 1.32, 1.53]
min :  0.0
max :  1.53
mean :  0.9500000000000001
fuzzy :  [(0.0, 0.0), (1.32, 0.3620689655172413), (1.53, 0.0)]
fuzzy :  None
pembilang :  0.4779310344827586
penyebut :  0.3620689655172413
rata-rata tertimbang :  1.32
 
[0.0, 1.09, 3.33]
min :  0.0
max :  3.33
mean :  1.4733333333333334
fuzzy :  [(0.0, 0.0), (1.09, 0.7398190045248869), (3.33, 0.0)]
fuzzy :  None
pembilang :  0.8064027149321268
penyebut :  0.7398190045248869
rata-rata tertimbang :  1.09
 
[0.0, 3.8, 6.25]
min :  0.0
max :  6.25
mean :  3.35
fuzzy :  [(0.0, 0.0), (3.8, 0.8448275862068967), (6.25, 0.0)]
fuzzy :  None
pembilang :  3.210344827586207
penyebut :  0.8448275862068967
rata-rata tertimbang :  3.8
 
[0.0, 1.26, 2.62]
min :  0.0
max :  2.62
mean :  1.2933333333333332
fuzzy :  [(0.0, 0.0), (2.62, 0.0), (1.26, 0.9742268041237114)]
fuzzy :  None
pembilang :  1.2275257731958764
penyebut :  0.9742268041237114
rata-rata tertimbang :  1.26
 
[0.0, 1.26, 2.93]
min :  0.0
max :  2.93
mean :  1.39666666666

[0.0, 0.0, 0.0]
median :  2
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0]
median :  2
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0]
median :  2
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0]
median :  2
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0]
median :  2
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0]
median :  2
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.

[0.0, 0.0, 0.0]
median :  2
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0]
median :  2
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0]
median :  2
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0]
median :  2
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0]
median :  2
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0]
median :  2
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.

[0.0, 0.0, 0.0]
median :  2
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0]
median :  2
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0]
median :  2
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0]
median :  2
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0]
median :  2
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0]
median :  2
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.

...

[0.0, 0.0, 0.0]
median :  2
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 1.0]
median :  2
min :  0.0
max :  1.0
mean :  0.3333333333333333
fuzzy :  [(0.0, 0.0), (1.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 1.0, 1.0]
median :  2
min :  0.0
max :  1.0
mean :  0.6666666666666666
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  2.0
penyebut :  3.0
rata-rata tertimbang :  1.0
 
[0.0, 0.0, 1.0]
median :  2
min :  0.0
max :  1.0
mean :  0.3333333333333333
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (1.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0]
median :  2
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 1.0, 1.0]
median :  2
min :  

[0.0, 1.0, 1.0]
median :  2
min :  0.0
max :  1.0
mean :  0.6666666666666666
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  2.0
penyebut :  3.0
rata-rata tertimbang :  1.0
 
[0.0, 0.0, 0.0]
median :  2
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0]
median :  2
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0]
median :  2
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 1.0, 1.0]
median :  2
min :  0.0
max :  1.0
mean :  0.6666666666666666
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  2.0
penyebut :  3.0
rata-rata tertimbang :  1.0
 
[0.0, 0.0, 0.0]
median :  2
min :  0.0
max :  0.0


[0.0, 1.0, 1.0]
median :  2
min :  0.0
max :  1.0
mean :  0.6666666666666666
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  2.0
penyebut :  3.0
rata-rata tertimbang :  1.0
 
[0.0, 1.0, 1.0]
median :  2
min :  0.0
max :  1.0
mean :  0.6666666666666666
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  2.0
penyebut :  3.0
rata-rata tertimbang :  1.0
 
[0.0, 0.0, 1.0]
median :  2
min :  0.0
max :  1.0
mean :  0.3333333333333333
fuzzy :  [(0.0, 0.0), (1.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 1.0, 1.0]
median :  2
min :  0.0
max :  1.0
mean :  0.6666666666666666
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  2.0
penyebut :  3.0
rata-rata tertimbang :  1.0
 
[0.0, 1.0, 1.0]
median :  2
min :  0.0
max :  1.0
mean :  0.6666666666666666
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  2.0
penyebut :  3.0
rata-rata tertimbang :  1.0
 
[0.0,

[0.0, 1.0, 1.0]
median :  2
min :  0.0
max :  1.0
mean :  0.6666666666666666
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  2.0
penyebut :  3.0
rata-rata tertimbang :  1.0
 
[0.0, 1.0, 1.0]
median :  2
min :  0.0
max :  1.0
mean :  0.6666666666666666
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  2.0
penyebut :  3.0
rata-rata tertimbang :  1.0
 
[0.0, 1.0, 1.0]
median :  2
min :  0.0
max :  1.0
mean :  0.6666666666666666
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  2.0
penyebut :  3.0
rata-rata tertimbang :  1.0
 
[0.0, 1.0, 1.0]
median :  2
min :  0.0
max :  1.0
mean :  0.6666666666666666
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  2.0
penyebut :  3.0
rata-rata tertimbang :  1.0
 
[0.0, 0.0, 1.0]
median :  2
min :  0.0
max :  1.0
mean :  0.3333333333333333
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (1.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0,

[0.0, 1.0, 1.0]
median :  2
min :  0.0
max :  1.0
mean :  0.6666666666666666
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  2.0
penyebut :  3.0
rata-rata tertimbang :  1.0
 
[0.0, 1.0, 1.0]
median :  2
min :  0.0
max :  1.0
mean :  0.6666666666666666
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  2.0
penyebut :  3.0
rata-rata tertimbang :  1.0
 
[0.0, 1.0, 1.0]
median :  2
min :  0.0
max :  1.0
mean :  0.6666666666666666
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  2.0
penyebut :  3.0
rata-rata tertimbang :  1.0
 
[0.0, 1.0, 1.0]
median :  2
min :  0.0
max :  1.0
mean :  0.6666666666666666
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  2.0
penyebut :  3.0
rata-rata tertimbang :  1.0
 
[0.0, 1.0, 1.0]
median :  2
min :  0.0
max :  1.0
mean :  0.6666666666666666
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  2.0
penyebut :  3.0
rata-rata tertimbang :  1.0
 
[0.0,

In [14]:
dummy

,Jitter (local),"Jitter (local, absolute)",Jitter (rap),Jitter (ppq5),Jitter (ddp),Shimmer (local),"Shimmer (local, dB)",Shimmer (apq3),Shimmer (apq5),Shimmer (apq11),...,Maximum pitch,Number of pulses,Number of periods,Mean period,Standard deviation of period,Fraction of locally unvoiced frames,Number of voice breaks,Degree of voice breaks,features 27,class
0,1.49,0.0,0.90,0.48,2.70,8.33,0.78,4.52,4.61,6.80,...,187.58,160.0,159.0,0.01,0.0,2.00,1.0,3.39,23.0,1.0
1,0.73,0.0,0.35,0.38,1.06,5.86,0.64,4.35,3.18,7.19,...,234.51,170.0,169.0,0.01,0.0,2.25,1.0,3.39,23.0,1.0
2,1.22,0.0,0.73,0.67,2.20,8.72,0.88,4.35,5.17,7.55,...,211.44,1431.0,1427.0,0.01,0.0,10.66,1.0,0.18,23.0,1.0
3,2.50,0.0,1.16,1.63,3.47,13.51,1.25,5.26,8.77,16.78,...,220.23,94.0,92.0,0.01,0.0,3.70,1.0,0.18,23.0,1.0
4,3.51,0.0,1.72,1.54,5.15,9.11,1.04,3.10,4.93,12.82,...,225.16,117.0,114.0,0.01,0.0,18.18,1.0,13.32,23.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1035,5.13,0.0,2.72,2.51,8.16,9.85,0.97,4.59,6.81,8.30,...,158.59,63.0,60.0,0.01,0.0,24.19,2.0,15.11,1.0,1.0
1036,1.76,0.0,0.93,0.69,2.43,7.57,0.74,2.72,4.32,11.93,...,143.80,47.0,44.0,0.01,0.0,46.05,1.0,25.79,1.0,1.0
1037,2.91,0.0,1.14,1.27,3.42,15.49,1.46,6.43,7.23,14.24,...,149.64,79.0,77.0,0.01,0.0,20.00,1.0,4.57,1.0,1.0
1038,3.79,0.0,1.79,1.48,5.36,15.70,1.37,6.37,11.09,18.82,...,451.37,75.0,70.0,0.01,0.0,20.00,3.0,19.58,1.0,1.0


In [15]:
dummy.describe()

,Jitter (local),"Jitter (local, absolute)",Jitter (rap),Jitter (ppq5),Jitter (ddp),Shimmer (local),"Shimmer (local, dB)",Shimmer (apq3),Shimmer (apq5),Shimmer (apq11),...,Maximum pitch,Number of pulses,Number of periods,Mean period,Standard deviation of period,Fraction of locally unvoiced frames,Number of voice breaks,Degree of voice breaks,features 27,class
count,1040.000000,1040.0,1040.000000,1040.000000,1040.000000,1040.000000,1040.000000,1040.000000,1040.000000,1040.000000,...,1040.000000,1040.000000,1040.000000,1.040000e+03,1040.00000,1040.000000,1040.000000,1040.000000,1040.000000,1040.0
mean,2.636154,0.0,1.235375,1.324154,3.677875,12.834087,1.183471,5.659010,7.837231,12.061942,...,234.109106,109.077885,105.516346,1.000000e-02,0.00001,28.853760,1.670192,18.242087,12.671154,1.0
std,1.756930,0.0,0.973702,1.135385,2.917399,5.415104,0.415066,2.991609,4.776004,5.833857,...,121.319222,149.677628,149.132460,1.700847e-16,0.00031,19.735973,1.430412,13.917506,15.762907,0.0
min,0.190000,0.0,0.060000,0.080000,0.190000,1.190000,0.100000,0.500000,0.710000,0.520000,...,85.540000,5.000000,4.000000,1.000000e-02,0.00000,0.160000,1.000000,0.180000,1.000000,1.0
25%,1.477500,0.0,0.600000,0.650000,1.765000,9.295000,0.940000,3.697500,5.150000,8.125000,...,143.210000,42.750000,40.000000,1.000000e-02,0.00000,11.082500,1.000000,6.720000,1.000000,1.0
50%,2.340000,0.0,1.030000,1.100000,3.040000,12.190000,1.175000,5.115000,6.925000,11.300000,...,195.590000,64.000000,62.000000,1.000000e-02,0.00000,27.400000,1.000000,15.520000,1.000000,1.0
75%,3.352500,0.0,1.600000,1.660000,4.772500,15.470000,1.400000,6.852500,9.235000,15.085000,...,263.357500,112.250000,109.000000,1.000000e-02,0.00000,43.150000,2.000000,25.942500,23.000000,1.0
max,14.380000,0.0,8.020000,13.540000,24.050000,41.140000,2.720000,25.820000,72.860000,42.000000,...,597.970000,1490.000000,1489.000000,1.000000e-02,0.01000,88.160000,12.000000,69.120000,55.000000,1.0


# Klasifikasi

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [17]:
x = dummy.values
y = df.iloc[:, -1].values

In [49]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=45)

In [50]:
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [51]:
classifier = GaussianNB()
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
print(cm)
print(accuracy)

[[135   0]
 [ 16 109]]
0.9384615384615385


In [56]:
classifier = MLPClassifier(hidden_layer_sizes=3, learning_rate_init=0.01, max_iter=100)
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
print(cm)
print(accuracy)

[[134   1]
 [  9 116]]
0.9615384615384616


C:\Users\User\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
